In [1]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
    config = 'configs.config_ILLAE_local'
#     config = 'configs.config_WomenInEngineering_local'
#     config = 'configs.config_Digits_local'    

In [2]:
### initialization ###

import importlib
print('config file is set to {}'.format(config))

imp=importlib.import_module('_imports')
importlib.reload(imp)

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_ILLAE_local


In [3]:
### reading class labels and concatenating features and original data ###

dfs=dict()

dfs['input']=None
dfs['labels'] = imp.pd.read_csv('{}data/{}_2_data_labels.csv'.format(c.directory['save'], c.project_name))
dfs['features']=util.construct_static_features(c)
if len(dfs['features']) == 0:
    dfs['features'] = imp.pd.DataFrame(dfs['labels'][c.id_name])
imp.pprint('labels = {}'.format(dfs['labels'].shape))
util.display_df(dfs['labels'].head())

imp.pprint('featuers = {}'.format(dfs['features'].shape))
util.display_df(dfs['features'].head())

'labels = (17354, 2)'


,tweet_id,label
0,637005633548632064,False
1,637007331574812673,True
2,637012950495031296,True
3,637012998473670656,True
4,637014738216443905,False


'featuers = (17354, 108)'


,tweet_id,friends_count,followers_count,favourites_count,statuses_count,profile_created_at_date,listed_count,verified,text,external_url,external_media_type,user_mentions,in_reply_to_user_id,hashtags_count,hashtags_sum,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,637005633548632064,34,946,78,1127,1586,51,0,131,1.0,0.0,0.0,0.0,2,14.5,21,92.84,92.33,1.00,95.81,21.0,23.81,61.90,38.10,9.52,0.00,0.0,0.00,0.0,0.0,0.0,9.52,4.76,19.05,9.52,4.76,0.00,0.0,9.52,4.76,4.76,4.76,0.0,0.0,4.76,4.76,0.0,0.0,0.0,0.0,14.29,0.0,0.0,4.76,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.52,4.76,0.00,4.76,0.00,0.0,0.00,14.29,0.0,4.76,0.00,4.76,0.00,4.76,0.00,0.0,0.0,0.0,0.0,4.76,0.0,4.76,0.0,0.0,0.0,33.33,4.76,0.0,4.76,0.0,0.0,0.0,0.0,0.0,4.76,0.0,19.05
1,637007331574812673,196,253,117,672,3175,11,0,139,0.0,0.0,1.0,0.0,1,19.0,21,92.84,99.00,68.01,99.00,10.5,28.57,80.95,42.86,14.29,9.52,0.0,9.52,0.0,0.0,0.0,4.76,9.52,14.29,0.00,4.76,4.76,0.0,4.76,9.52,4.76,0.00,0.0,0.0,9.52,9.52,0.0,0.0,0.0,0.0,14.29,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.81,9.52,9.52,4.76,4.76,0.0,4.76,4.76,0.0,23.81,4.76,0.00,23.81,9.52,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,23.81,4.76,0.0,0.00,0.0,0.0,0.0,0.0,0.0,4.76,0.0,14.29
2,637012950495031296,1287,13,2,3,2316,2,0,43,0.0,1.0,0.0,0.0,1,19.0,6,92.84,50.00,1.00,25.77,6.0,33.33,16.67,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,16.67,0.0,16.67,0.0,0.0,0.0,66.67,16.67,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,50.00
3,637012998473670656,464,494,2341,8345,2465,2,0,67,0.0,1.0,1.0,1.0,1,19.0,9,92.84,50.00,1.00,25.77,9.0,33.33,11.11,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,11.11,0.0,11.11,0.0,0.0,0.0,77.78,11.11,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,66.67
4,637014738216443905,416,206,16,85,2904,7,0,136,0.0,1.0,1.0,0.0,2,13.5,19,92.84,70.08,50.35,25.77,9.5,36.84,47.37,10.53,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,5.26,0.00,0.00,5.26,0.00,0.0,5.26,5.26,5.26,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,5.26,0.0,0.0,0.00,0.0,5.26,0.0,5.26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.53,5.26,0.00,5.26,0.00,0.0,10.53,5.26,0.0,21.05,10.53,0.00,10.53,5.26,5.26,0.0,0.0,0.0,0.0,5.26,0.0,5.26,0.0,0.0,0.0,36.84,10.53,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,26.32


In [4]:
### reading cv ids for cv folds ###

outer_ids = imp.pd.read_csv('{}{}'.format(c.directory['save'], 'cv_ids/outer_0.csv'))
outer_ids.rename(columns={'id': c.id_name}, inplace=True)

inner_ids = list()
for i in range(c.inner):
  inner_ids.append(imp.pd.read_csv('{}cv_ids/inner_0_{}.csv'.format(c.directory['save'], i)))

imp.pprint('Number of cv = {}'.format(len(inner_ids)))

'Number of cv = 5'


In [5]:
### for running classification ###

def for_loop_cv(experiment, df_features, df_labels, dynamics):

    cv=experiment['cv']
    training = imp.copy.deepcopy(inner_ids)
    del training[cv]
    training_ids=imp.pd.DataFrame()
    for item in training:
        training_ids=imp.pd.concat([training_ids, item], ignore_index=True, axis=0)
    training_ids.rename(columns={'id':c.id_name}, inplace=True)

    validation_ids = imp.copy.deepcopy(inner_ids[cv])
    validation_ids.rename(columns={'id':c.id_name}, inplace=True)


    for feature in c.select_features['dynamic']:
        dynamic=dynamics[c.features['dynamic'][feature]['type']]
        print(c.features['dynamic'][feature]['type'])
        df_dyanmic=dynamic.retrieve_features(c, experiment, feature)

#         util.display(df_dyanmic.head())
        df_features=imp.pd.merge(df_features, df_dyanmic, on=c.id_name, how='right')
        
    result_training = util.data_prepropressing(training_ids, 
                                               df_features, 
                                               df_labels,
                                               c
                                              )

    result_validation = util.data_prepropressing(validation_ids,
                                                 df_features, 
                                                 df_labels, 
                                                 c,
                                                 result_training['imputer'], 
                                                 result_training['scaler']
                                                )    
    
    classifier = imp.SVC(C=experiment['cost'] if 'cost' in experiment else 1,
                         kernel=experiment['kernel'] if 'kernel' in experiment else 'linear',
                         gamma=experiment['gamma'] if 'gamma' in experiment else 1,
                         coef0=experiment['coef0'] if 'coef0' in experiment else 1,
                         degree=experiment['degree'] if 'degree' in experiment else 1,
                         shrinking=True,
                         max_iter=c.max_iter_optimization,
                         decision_function_shape='ovo',
                         cache_size=c.cache_size,
                         random_state=c.seed
                        )
    
    classifier.fit(result_training['features'], list(result_training['labels']['label']))  
    array=imp.np.array(result_validation['features'])
    if len(result_validation['features'])==1:
        array=array.reshape(1, -1)
        
    predictions=classifier.predict(array)
    return dict(
        experiment=experiment
        , pred=list(predictions)
        , true=list(result_validation['labels']['label'])
    )

In [6]:
### getting the parameter grid for the experiments ###

dfs['experiments']=imp.pd.read_csv('{}experiments/{}/1_parameters.csv'.format(c.directory['save'],
                                                                              util.experiment_name(c)))
experiments_ids= list(range(c.index_start, len(dfs['experiments']), c.index_step))
dfs['experiments']=dfs['experiments'].iloc[experiments_ids]

imp.pprint('number of experiments = {}'.format(len(dfs['experiments'])))
util.display_df(dfs['experiments'].head(11))

'number of experiments = 150'


,coef0,cost,cv,degree,full_text_LDA_num_topics,full_text_LDA_topic_cutoff_threshold,full_text_no_above,full_text_no_below,gamma,hashtags_community_feature_type,hashtags_community_graph_type,kernel,lda_hashtags_LDA_num_topics,lda_hashtags_LDA_topic_cutoff_threshold,lda_hashtags_no_above,lda_hashtags_no_below
0,1.0,1,0,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1
1,1.0,1,1,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1
2,1.0,1,2,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1
3,1.0,1,3,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1
4,1.0,1,4,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1
5,1.0,10,0,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1
6,1.0,10,1,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1
7,1.0,10,2,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1
8,1.0,10,3,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1
9,1.0,10,4,1.0,20,0,1,1,1.00,weighted,weighted,linear,20,0,1,1


In [7]:
### loading dynamic feature modules ###:

dynamics=dict()
for feature in c.select_features['dynamic']:
    dynamics[c.features['dynamic'][feature]['type']]=importlib.import_module('dynamic_features.{}'.format(c.features['dynamic'][feature]['type']))
    importlib.reload(dynamics[c.features['dynamic'][feature]['type']])

In [8]:
### debugging purposes-1 (Experimental Setup) ###

# params=dict(
#     df_features=dfs['features'], 
#     df_labels=dfs['labels'], 
#     dynamics=dynamics
# )

# df_features=dfs['features']
# experiment=[exp for exp in dfs['experiments'].to_dict('records')][0]
# print('_'.join(list(map(str, list(experiment.values())))))
# experiment

In [9]:
### debugging purposes-2 (Feature Construction) ###

# cv=experiment['cv']
# training = imp.copy.deepcopy(inner_ids)
# del training[cv]
# training_ids=imp.pd.DataFrame()
# for item in training:
#     training_ids=imp.pd.concat([training_ids, item], ignore_index=True, axis=0)
# training_ids.rename(columns={'id':c.id_name}, inplace=True)

# validation_ids = imp.copy.deepcopy(inner_ids[cv])
# validation_ids.rename(columns={'id':c.id_name}, inplace=True)


# for feature in c.select_features['dynamic']:
#     dynamic=dynamics[c.features['dynamic'][feature]['type']]
#     print(c.features['dynamic'][feature]['type'])
#     df_dyanmic=dynamic.retrieve_features(c, experiment, feature)

#     util.display(df_dyanmic.head())
#     df_features=imp.pd.merge(df_features, df_dyanmic, on=c.id_name, how='right')

# print(df_features.shape)
# df_features    

In [10]:
### debugging purposes-3 ###

# for_loop_cv(experiment, **params)

# OR 

# list(map(imp.functools.partial(for_loop_cv, **params), [experiment for experiment in dfs['experiments'].to_dict('records')]))

In [11]:
### running the experiments in parallel ###

result_cvs=imp.Parallel(n_jobs=c.num_cores
                    , prefer="processes"
                    , verbose=3
                   ) (imp.delayed(for_loop_cv)(experiment, 
                                               dfs['features'], 
                                               dfs['labels'], 
                                               dynamics
                                              ) 
                      for experiment in dfs['experiments'].to_dict('records'))

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done   2 tasks      | elapsed:  1.3min
/anaconda3/envs/RT/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=15)]: Done  98 tasks      | elapsed: 12.0min
[Parallel(n_jobs=15)]: Done 150 out of 150 | elapsed: 15.4min finished


In [12]:
# saving the results ###

util.mkdir('{}tmp/'.format(c.directory['save']))

dbfile = open('{}tmp/{}_db_result_cvs_{}_from_{}.pickle'.format(c.directory['save'],
                                                                   util.experiment_name(c),
                                                                   c.index_start+1,
                                                                   c.index_step
                                                                  ), 'wb') 
imp.pickle.dump(result_cvs, dbfile)
dbfile.close()

folder /Users/habibkarbasian/Documents/_Courses/_Datasets/_Retweetability/_ILLAE/tmp/ already exists!
